In [3]:
from transformers import RobertaModel, PreTrainedModel
from torch import nn
class CustomROBERTAModel(PreTrainedModel):
      def __init__(self, config, transformer_model_name_or_path, num_feats):
            super(CustomROBERTAModel, self).__init__(config)
            self.roberta = RobertaModel.from_pretrained(
                  transformer_model_name_or_path,
                  config=config
            )
            ### New layers:
            self.num_feats = num_feats
            self.output_layers = []
            for _ in range(self.num_feats):
                  self.output_layers.append(
                        nn.Sequential(
                              nn.Linear(config.hidden_size, 512),
                              nn.GELU(),
                              nn.Linear(512, 256),
                              nn.GELU(),
                              nn.Linear(256, 128),
                              nn.GELU(),
                              nn.Linear(128, 64),
                              nn.GELU(),
                              nn.Linear(64, 32),
                              nn.GELU(),
                              nn.Linear(32, 9),
                              nn.Softmax(),
                        ).to('cuda')
                  )

      def forward(self, **inputs):
            roberta_outputs = self.roberta(**inputs)
            outputs = []
            for i in range(self.num_feats):
                  outputs.append(
                        self.output_layers[i](roberta_outputs.pooler_output)[0]
                  )
            logits = outputs
            return logits

      def _init_weights(self, module):
            self.bert._init_weights(module)

In [4]:
from transformers import AutoConfig, AutoTokenizer
import torch
base_model = 'roberta-large'
checkpoint = '/home/yuuhanase/FPTU/EXE101/PaperClipAI_EnglishGrading/EnglishGradingModel'
config = AutoConfig.from_pretrained(base_model)
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = CustomROBERTAModel(
    config=config, 
    transformer_model_name_or_path=base_model, 
    num_feats=6
).to("cuda") ## can be gpu
# model = CustomROBERTAModel.from_pretrained(
#     checkpoint, 
#     config=config, 
#     transformer_model_name_or_path=base_model,
#     num_feats=6
# )

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
text = "Hello"
encoded_input = tokenizer(text, return_tensors='pt').to("cuda")
output = model(**encoded_input)
output

/home/yuuhanase/miniconda3/lib/python3.11/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


[tensor([[0.1226, 0.1213, 0.1031, 0.1220, 0.0926, 0.1157, 0.0938, 0.1252, 0.1037]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor([[0.1172, 0.0983, 0.1352, 0.1339, 0.0981, 0.1073, 0.0976, 0.0954, 0.1171]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor([[0.1154, 0.1133, 0.1052, 0.1126, 0.1049, 0.1033, 0.0999, 0.1227, 0.1226]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor([[0.1038, 0.1091, 0.1067, 0.1053, 0.1242, 0.1250, 0.1276, 0.1055, 0.0927]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor([[0.1023, 0.1231, 0.1073, 0.1098, 0.1221, 0.1122, 0.0965, 0.1054, 0.1213]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor([[0.1016, 0.1342, 0.1001, 0.1168, 0.1270, 0.1106, 0.1010, 0.1011, 0.1076]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>)]

In [3]:
from transformers import RobertaTokenizer, RobertaModel, AutoConfig
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
config = AutoConfig.from_pretrained('roberta-large')
model = RobertaModel.from_pretrained('roberta-large')
text = "Hello"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
output

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0384, -0.0209, -0.0141,  ..., -0.0523,  0.0802,  0.0632],
         [ 0.3952,  0.0993, -0.6829,  ...,  0.1277, -0.4668,  0.4225],
         [-0.0097, -0.0010,  0.0426,  ..., -0.0275,  0.0668,  0.0367]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.8011, -0.4364,  0.2403,  ..., -0.4839,  0.1486, -0.3779]],
       grad_fn=<TanhBackward0>), hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None)

In [4]:
output.

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0384, -0.0209, -0.0141,  ..., -0.0523,  0.0802,  0.0632],
         [ 0.3952,  0.0993, -0.6829,  ...,  0.1277, -0.4668,  0.4225],
         [-0.0097, -0.0010,  0.0426,  ..., -0.0275,  0.0668,  0.0367]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.8011, -0.4364,  0.2403,  ..., -0.4839,  0.1486, -0.3779]],
       grad_fn=<TanhBackward0>), hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None)

In [10]:
inp_text = "When a problem is a change you have to let it do the best on you no matter what is happening it can change your mind. sometimes you need to wake up and look what is around you because problems are the best way to change what you want to change along time ago. A problem is a change for you because it can make you see different and help you to understand how tings wok. First of all it can make you see different then the others. For example i remember that when i came to the United States i think that nothing was going to change me because i think that nothing was going to change me because everything was different that my country and then i realist that wrong because a problem may change you but sometimes can not change the way it is, but i remember that i was really shy but i think that change a lot because sometimes my problems make me think that there is more thing that i never see in my life but i just need to see it from a different way and dont let nothing happened and ruing the change that i want to make because of just a problem. For example i think that nothing was going to change me and that i dont need to be shy anymore became i need to start seeing everything in a different ways because you can get mad at every one but you need to know what is going to happened after, people may see you different but the only way that you know how to change is to do the best and don't let nothing or not body to change nothing about you. The way you want to change not one have that and can't do nothing about it because is your choice and your problems and you can decide what to do with it. second of all can help you to understand how things work. For instance my mom have a lot of problems but she have faith when she is around people, my mom is scare of high and i'm not scare of high i did not understand why my mos is scare of high and in not scare of high and every time i see my mom in a airplane it make me laugh because she is scare and is funny, but i see it from a different way and i like the high but also she have to understand that hoe things work in other people because it can no be the same as you. For example i think that my mom and me are different because we are and i have to understand that she does not like high and i need to understand that. to help someone to understand how things work you need to start to see how things work in that persons life. A problem is a change for you and can make you a different and help you to understand. Everyone has a different opinion and a different was to understand then others. everyone can see the different opinion and what other people think."
#2.5,2.5,3.0,2.0,2.0,2.5
tokenized_input = tokenizer(inp_text, return_tensors='pt', truncation=True)#.to(torch.device("cuda"))
output = model(**tokenized_input)[0]
feats = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
result = {}
for i in range(6):
    result[feats[i]] = output[i].item()*5.0
result

{'cohesion': 2.383091002702713,
 'syntax': 2.0666779577732086,
 'vocabulary': 2.884359359741211,
 'phraseology': 2.844240665435791,
 'grammar': 2.9789209365844727,
 'conventions': 2.2683973610401154}

In [10]:
from transformers import Trainer
from torch import nn
class CustomTrainer(Trainer):
    def __int__(self, *args, **kwargs):
        super().__int__(*args, **kwargs)
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        # forward pass
        logits = model(**inputs)
        loss_fct = nn.MSELoss()
        loss = loss_fct(logits, labels)
        return (loss, logits) if return_outputs else loss

In [ ]:
trainer